# Benchmarks

In [1]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np
# this is the convenience function
from autokoopman import auto_koopman
# for a complete example, let's create an example dataset using an included benchmark system
from autokoopman.benchmark import bio2, fhn, lalo20, prde20, robe21, spring, pendulum
from glop import Glop
import random
import copy

from sklearn.metrics import mean_squared_error
import statistics
import os
import csv
import time

In [2]:
benches = [bio2.Bio2(), fhn.FitzHughNagumo(), lalo20.LaubLoomis(), pendulum.PendulumWithInput(beta=0.05), prde20.ProdDestr(), robe21.RobBench(), spring.Spring()]

In [3]:
def get_training_data(bench, param_dict, iterat):
    init_states = get_init_states(bench, param_dict["train_size"], iterat)
    if bench._input_vars:
        params = np.random.rand(param_dict["train_size"], 3) * 2 - 1
        steps = [make_input_step(*p, bench.teval) for p in params]
        training_data = bench.solve_ivps(initial_states=init_states, inputs=steps, teval=bench.teval)
    else:
        training_data = bench.solve_ivps(initial_states=init_states,tspan=[0.0, 10.0], 
                                         sampling_period=param_dict["samp_period"])
        
    return training_data

In [4]:
def get_init_states(bench, size, iterat=0):
    if hasattr(bench, 'init_constrs'):
        init_states = []
        for i in range(size):
            init_state_dict = glop_init_states(bench, i+iterat)
            init_state = []
            for name in bench.names:
                init_state.append(init_state_dict[name])
            init_states.append(init_state)
        init_states = np.array(init_states)  
    else:
        init_states = np.random.uniform(low=bench.init_set_low, 
                    high=bench.init_set_high, size=(size, len(bench.names)))
        
    print(init_states)
    return init_states

In [5]:
def glop_init_states(bench, seed):    
    constrs = []
    for constr in bench.init_constrs:
        constrs.append(constr)
    for i, (name, init_low, init_high) in enumerate(zip(bench.names, bench.init_set_low, bench.init_set_high)):
        low_constr = f"{name} >= {init_low}"
        high_constr = f"{name} <= {init_high}"
        constrs.extend([low_constr, high_constr])
        
    glop = Glop(bench.names, constrs)
    pop_item = random.randrange(len(bench.names))
    names, init_set_low, init_set_high = copy.deepcopy(bench.names), copy.deepcopy(bench.init_set_low), copy.deepcopy(bench.init_set_high)
    names.pop(pop_item)
    init_set_low.pop(pop_item)
    init_set_high.pop(pop_item)
    for i, (name, init_low, init_high) in enumerate(zip(names, init_set_low, init_set_high)):
        glop.add_tend_value_obj_fn(name, [init_low, init_high], seed+i)
    
    glop.minimize()

    sol_dict = glop.get_all_sols()    
    return sol_dict

In [6]:
def get_trajectories(bench, iv, samp_period):
    # get the model from the experiment results
    model = experiment_results['tuned_model']

    if bench._input_vars:
        test_inp = np.sin(np.linspace(0, 10, 200))

        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            inputs=test_inp,
            teval=bench.teval,
        )
        
    else:
        # simulate using the learned model
        trajectory = model.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=samp_period
        )
        # simulate the ground truth for comparison
        true_trajectory = bench.solve_ivp(
            initial_state=iv,
            tspan=(0.0, 10.0),
            sampling_period=samp_period
        )
    
    return trajectory, true_trajectory

In [7]:
def test_trajectories(bench, num_tests, samp_period, iterat=0):
    mses = []
    mses_dim = [[] for x in range(len(bench.names))]
    for j in range(num_tests):
        iv = get_init_states(bench, 1, iterat+j+1000)[0]
        trajectory, true_trajectory = get_trajectories(bench, iv, samp_period)
        mse = mean_squared_error(trajectory.states.T, true_trajectory.states.T)
        mses.append(mse)
        
        for traj_dim, (trajectory, true_trajectory) in enumerate(zip(trajectory.states.T, true_trajectory.states.T)):
            mse = mean_squared_error(trajectory, true_trajectory)
            mses_dim[traj_dim].append(mse)
            
    return mses, mses_dim

In [8]:
def make_input_step(duty, on_amplitude, off_amplitude, teval):
    """produce a step response input signal for the pendulum"""
    length = len(teval)
    inp = np.zeros((length,))
    phase_idx = int(length * duty)
    inp[:phase_idx] = on_amplitude
    inp[phase_idx:] = off_amplitude
    return inp

In [9]:
def store_data(bench_name, dim, param_name, param_values, train_times, all_mses, all_mses_dim):
    if not os.path.exists('data'):
        os.makedirs('data')
        
    with open(f'data/{bench_name}', 'a') as f:
        writer = csv.writer(f)
        row = [param_name, "train_time", "Avg mse"]
        for i in range(dim):
              row.append(f'Avg mse dim {i+1}')
        writer.writerow(row)
              
        for param_value, train_time, mse, mses_dim in zip(param_values, train_times, all_mses, all_mses_dim):
              row = [param_value, train_time, mse]
              for mses in mses_dim:
                  row.append(mses)
              writer.writerow(row)
        writer.writerow([])

In [10]:
def plot(trajectory, true_trajectory):
    plt.figure(figsize=(10, 6))
    # plot the results
    plt.plot(trajectory.states.T[0], trajectory.states.T[1],label='Trajectory Prediction')
    plt.plot(true_trajectory.states.T[0], true_trajectory.states.T[1],label='Ground Truth')

    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid()
    plt.legend()
    plt.title("Bio2 Test Trajectory Plot")
    plt.show()

In [11]:
all_param_values = {
    "train_size":[5, 10, 25, 50, 75, 100, 125, 150, 175, 200],
    "samp_period":[0.01, 0.025, 0.05, 0.1, 0.2, 0.25, 0.5, 1, 2],
    "obs_type":["rff","quadratic", "id"],
    "opt":["grid", "monte-carlo"],
    "n_obs":[5, 10, 25, 50, 100, 200, 300, 400, 500, 1000],
    "grid_param_slices":[5, 10, 25, 50, 100],
    "n_splits":[2, 5, 10],
    "rank":[(1, 200, 40)]
}

In [12]:
for bench in benches:
    for param, param_values in all_param_values.items():
        all_mses = []
        all_mses_dim = []
        times = []
        param_dict = {"train_size":10,"samp_period":0.1,"obs_type":"rff","opt":"grid","n_obs":200,
                      "grid_param_slices":5,"n_splits":5,"rank":(1, 200, 40)}
        for i, param_value in enumerate(param_values):
            np.random.seed(0)
            param_dict[param] = param_value
            start = time.time()
            mses = [-1]
            iterat = 0
            while mses==[-1] and iterat < 5:
                iterat +=1
                all_mses_dim.append([])
                try:
                    # generate training data
                    print("traininggggg:")
                    training_data = get_training_data(bench, param_dict, iterat)
                    # learn model from data
                    experiment_results = auto_koopman(
                        training_data,          # list of trajectories
                        sampling_period=param_dict["samp_period"],    # sampling period of trajectory snapshots
                        obs_type=param_dict["obs_type"],         # use Random Fourier Features Observables
                        opt=param_dict["opt"],             # grid search to find best hyperparameters
                        n_obs=param_dict["n_obs"],              # maximum number of observables to try
                        max_opt_iter=200,       # maximum number of optimization iterations
                        grid_param_slices=param_dict["grid_param_slices"],# for grid search, number of slices for each parameter
                        n_splits=param_dict["n_splits"],             # k-folds validation for tuning, helps stabilize the scoring
                        rank=param_dict["rank"]       # rank range (start, stop, step) DMD hyperparameter
                    )
            
                    print("testing:")
                    mses, mses_dim = test_trajectories(bench, 10, param_dict["samp_period"], iterat)
                except ValueError:
                    print("mses: ", mses)
                    print("benchmark did not compute for this setting")
                                
            end = time.time()       
            times.append(round(end - start, 3))
            all_mses_dim.append([])
            print("The average mean square error is ", statistics.mean(mses))
            all_mses.append(statistics.mean(mses))
            for traj_dim, mses in enumerate(mses_dim):
                all_mses_dim[i].append(statistics.mean(mses))
                print(f"The average mean square error for dim {traj_dim+1} is", statistics.mean(mses))

        store_data(bench.name, len(bench.names), param, param_values, times, all_mses, all_mses_dim)

traininggggg:
[[9.56718212 0.43281788 0.        ]
 [9.5        0.47801714 0.02198286]
 [9.61898231 0.11802404 0.26299364]
 [9.57052511 0.11802404 0.31145085]
 [9.81145085 0.18854915 0.        ]]


  4%|█▊                                          | 1/25 [00:00<00:12,  1.96it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
 32%|██████████████                              | 8/25 [00:04<00:09,  1.78it/s]/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
 36%|███████████████▊                            | 9/25 [00:05<00:09,  1.72it/s]/usr/local/anaconda3/lib/python3.8/si

testing:
[[9.5        0.39832548 0.10167452]]
[[9.7604742 0.        0.2395258]]
[[9.74867071 0.04076684 0.21056245]]
[[9.71056245 0.2415552  0.04788235]]
[[9.7415552  0.08513482 0.17330998]]
[[9.5        0.17330998 0.32669002]]
[[9.92796969 0.07203031 0.        ]]
[[9.75991191 0.14999188 0.09009621]]
[[9.59009621 0.07440105 0.33550274]]
[[9.5        0.33550274 0.16449726]]
The average mean square error is  0.5449716208154313
The average mean square error for dim 1 is 1.028836246193746
The average mean square error for dim 2 is 0.4404305574407777
The average mean square error for dim 3 is 0.16564805881177036
traininggggg:
[[9.5        0.06718212 0.43281788]
 [9.5        0.47801714 0.02198286]
 [9.61898231 0.11802404 0.26299364]
 [9.57052511 0.11802404 0.31145085]
 [9.81145085 0.18854915 0.        ]
 [9.5        0.39667004 0.10332996]
 [9.72473069 0.16191638 0.11335293]
 [9.61335293 0.15514339 0.23150368]
 [9.73150368 0.         0.26849632]
 [9.7857013  0.         0.2142987 ]]


  4%|█▊                                          | 1/25 [00:00<00:18,  1.29it/s]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
  8%|███▌                                        | 2/25 [00:01<00:18,  1.26it/s]/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
 12%|█████▎                                      | 3/25 [00:02<00:17,  1.23it/s]/usr/local/anaconda3/lib/python3.8/si

testing:
[[9.89832548 0.10167452 0.        ]]
[[9.7604742 0.        0.2395258]]
[[9.74867071 0.04076684 0.21056245]]
[[9.71056245 0.2415552  0.04788235]]
[[9.7415552  0.08513482 0.17330998]]
[[9.5        0.17330998 0.32669002]]
[[9.92796969 0.07203031 0.        ]]
[[9.75991191 0.14999188 0.09009621]]
[[9.59009621 0.07440105 0.33550274]]
[[9.5        0.33550274 0.16449726]]
The average mean square error is  12.468463241128113
The average mean square error for dim 1 is 17.543921686140585
The average mean square error for dim 2 is 18.666920039459708
The average mean square error for dim 3 is 1.1945479977840459
traininggggg:
[[9.5        0.06718212 0.43281788]
 [9.5        0.47801714 0.02198286]
 [9.61898231 0.11802404 0.26299364]
 [9.57052511 0.11802404 0.31145085]
 [9.81145085 0.18854915 0.        ]
 [9.5        0.39667004 0.10332996]
 [9.72473069 0.16191638 0.11335293]
 [9.61335293 0.15514339 0.23150368]
 [9.73150368 0.         0.26849632]
 [9.7857013  0.         0.2142987 ]
 [9.7261897

...

testing:
[[1.19306908 0.49205981 0.         0.        ]]
[[1.2050023  0.49119698 0.         0.        ]]
[[1.19870612 0.49834676 0.         0.        ]]
[[1.20305836 0.50598638 0.         0.        ]]
[[1.20227551 0.49193044 0.         0.        ]]
[[1.1949474  0.49648329 0.         0.        ]]
[[1.20595994 0.49221225 0.         0.        ]]
[[1.19298065 0.49063481 0.         0.        ]]
[[1.20213986 0.49501686 0.         0.        ]]
[[1.20433914 0.50085168 0.         0.        ]]
The average mean square error is  1.399955179592411e-06
The average mean square error for dim 1 is 6.182129006396887e-07
The average mean square error for dim 2 is 8.427947610428944e-07
The average mean square error for dim 3 is 1.6079115167852975e-06
The average mean square error for dim 4 is 2.5309015399017635e-06
traininggggg:
[[1.20097627 0.50430379 0.         0.        ]
 [1.1984731  0.50291788 0.         0.        ]
 [1.20927326 0.49766883 0.         0.        ]
 [1.20136089 0.50851193 0.         0. 

  4%|█▊                                          | 1/25 [00:01<00:38,  1.59s/it]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
 12%|█████▎                                      | 3/25 [00:05<00:37,  1.69s/it]/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/interpolate/_interpolate.py:641: RuntimeWarning: overflow encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
 16%|███████                                     | 4/25 [00:06<00:37,  1.80s/it]/usr/local/anaconda3/lib/python3.8/si

testing:
[[1.19777826 0.49731083 0.         0.        ]]
[[1.20655962 0.49851966 0.         0.        ]]
[[1.19526655 0.5087619  0.         0.        ]]
[[1.1975687  0.49485799 0.         0.        ]]
[[1.2047344  0.49909047 0.         0.        ]]
[[1.20758634 0.49189222 0.         0.        ]]
[[1.19985998 0.49046229 0.         0.        ]]
[[1.19688416 0.4979247  0.         0.        ]]
[[1.20197168 0.49497231 0.         0.        ]]
[[1.19396174 0.50362221 0.         0.        ]]
The average mean square error is  1.2759928188338594e-06
The average mean square error for dim 1 is 1.0641024657818898e-06
The average mean square error for dim 2 is 4.907437361038182e-07
The average mean square error for dim 3 is 2.201931276950478e-06
The average mean square error for dim 4 is 1.347193796499252e-06
traininggggg:
[[1.20097627 0.50430379 0.         0.        ]
 [1.1984731  0.50291788 0.         0.        ]
 [1.20927326 0.49766883 0.         0.        ]
 [1.20136089 0.50851193 0.         0. 

  4%|█▊                                          | 1/25 [00:00<00:19,  1.24it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2559: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
  8%|███▌                                        | 2/25 [00:01<00:18,  1.25it/s]/Users/b6062805/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2560: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: overflow encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
/usr/local/anaconda3/lib/python3.8/site-packages/autokoopman/estimator/koopman.py:83: RuntimeWarning: invalid value encountered in matmul
  return np.real(self._A @ obs.T).flatten()[: len(x)]
 12%|█████▎                                      | 3/25 [00:02<00:18,  1.20it/s]/usr/local/anaconda3/lib/python3.8/site-

testing:
[[1.19306908 0.49205981 0.         0.        ]]
[[1.2050023  0.49119698 0.         0.        ]]
[[1.19870612 0.49834676 0.         0.        ]]
[[1.20305836 0.50598638 0.         0.        ]]
[[1.20227551 0.49193044 0.         0.        ]]
[[1.1949474  0.49648329 0.         0.        ]]
[[1.20595994 0.49221225 0.         0.        ]]
[[1.19298065 0.49063481 0.         0.        ]]
[[1.20213986 0.49501686 0.         0.        ]]
[[1.20433914 0.50085168 0.         0.        ]]
The average mean square error is  1.399955179592411e-06
The average mean square error for dim 1 is 6.182129006396887e-07
The average mean square error for dim 2 is 8.427947610428944e-07
The average mean square error for dim 3 is 1.6079115167852975e-06
The average mean square error for dim 4 is 2.5309015399017635e-06
